In [2]:
import pandas as pd

import torch
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer, util

DOLLY_MODEL = "databricks/dolly-v2-3b"
EMBEDDING_MODEL = "all-mpnet-base-v2"


In [3]:
generate_text = pipeline(model=DOLLY_MODEL, torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map=None, return_full_text=True)
# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)
embedding_model = SentenceTransformer(EMBEDDING_MODEL)


In [4]:
df_text_chunks = pd.read_feather("../data/paper_extracts_embed.feature")
df_text_chunks.head()


,text_chunk,title,embeddings
0,Texture Synthesis Using Convolutional NeuralNe...,Texture Synthesis Using Convolutional Neural N...,"[0.01643213, 0.08130997, -0.051866777, 0.07214..."
1,2 Convolutional neural network We use the VGG...,Texture Synthesis Using Convolutional Neural N...,"[0.004942025, -0.0018913074, -6.292959e-05, 0...."
2,3 different features. These feature correlati...,Texture Synthesis Using Convolutional Neural N...,"[0.040369675, 0.036883876, -0.026579427, 0.090..."
3,conv1_1pool1pool2pool3pool4originalPortilla &...,Texture Synthesis Using Convolutional Neural N...,"[-0.005054486, -0.011686498, -0.058328077, 0.0..."
4,6 Classification performance 1.00.80.60.4 top...,Texture Synthesis Using Convolutional Neural N...,"[0.022617005, -0.047514006, -0.008524228, 0.05..."


In [5]:
query = "What is the goal of visual texture synthesis?"
query_emb = embedding_model.encode(query)
doc_emb = df_text_chunks["embeddings"]
docs = df_text_chunks["text_chunk"]

scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)


/home/thusitha/miniconda3/envs/dolly/lib/python3.10/site-packages/sentence_transformers/util.py:61: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  b = torch.tensor(b)


'\x0cconv1_1pool1pool2pool3pool4originalPortilla & Simoncelli Figure 2: Generated stimuli. Each row corresponds to a different processing stage in the network.When only constraining the texture representation on the lowest layer, the synthesised textures havelittle structure (first row). With increasing number of layers on which we match the texture representation we find that we generate images with increasing degree of naturalness (rows 25; labelson the left indicate the top-most layer included). The source textures in the first three columns werepreviously used by Portilla and Simoncelli [21]. For better comparison we also show their results(last row). The last column shows textures generated from a non-texture image to give a betterintuition about how the texture model represents image information.5 \x0cA ~1k parameters ~10k parameters ~177k parameters ~852k parameters original B conv1 conv2 conv3 conv4 conv5 C conv1_1 pool1 pool2 pool3 pool4\n\nTexture Synthesis Using Convolutiona

In [6]:
context_chunks = "\n\n".join([doc_score_pairs[i][0] for i in range(3)])
print(llm_context_chain.predict(instruction=query, context=context_chunks).lstrip())

KeyboardInterrupt: 